In [1]:
import tensorflow as tf

C:\ProgramData\Anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
C:\ProgramData\Anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
C:\ProgramData\Anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
C:\ProgramData\Anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:519: FutureWarning: Passi

## 변수 저장

In [3]:
v = tf.Variable(5)
w = tf.Variable(10, name='www')
updateV = v.assign_add(1)
updateW = w.assign_add(1)
init = tf.global_variables_initializer()
saver = tf.train.Saver()

with tf.Session() as sess:
    sess.run(init)
    for epoch in range(1000):
        sess.run([updateV, updateW])
    saver.save(sess, "/tmp/variables_final.dat")
    finalV = v.eval()
    finalW = w.eval()
    print(finalV)
    print(finalW)

1005
1010


In [5]:
#위에서 업데이트 한 내용은 세션 자체에 저장되어서 save된 상태이다.
#v 자체는 단순 노드임.
init = tf.global_variables_initializer()
with tf.Session() as sess:
    sess.run(init)
    print(v.eval())

5


In [6]:
with tf.Session() as sess:
    saver.restore(sess, "/tmp/variables_final.dat")
    finalV = v.eval()
    finalW = w.eval()
    print(finalV)
    print(finalW)

Instructions for updating:
Use standard file APIs to check for files with this prefix.
INFO:tensorflow:Restoring parameters from /tmp/variables_final.dat
1005
1010


## 변수 공유
공유하고자 하는 변수들에 별도의 scope를 부여하고
공유 여부를 지정하는 reuse 플래그를 사용하여 공유.
tf.variable_scope("namespace", reuse=True)
v = tf.get_variable(name, shape, dtype, initializer)

In [22]:
with tf.variable_scope("mynamespace", reuse=True):
    print(tf.get_variable_scope().name)
    x = tf.get_variable("x", [1])
    y = tf.get_variable("y", [1])
# reuse가 False이면 변수가 없는경우 새로 생성, 있으면 오류
# True이면 변수가 없는 경우 오류, 있는경우 가져다가 사용
with tf.variable_scope("mynamespace"):
    tf.get_variable_scope().reuse_variables() #reuse 플래그를 true로 만들어준다.
    x2 = tf.get_variable("x", [1])
    y2 = tf.get_variable("y", [1])
    
assert x == x2 and y == y2

mynamespace


In [30]:
with tf.variable_scope("namespace", initializer=tf.constant_initializer(6),reuse=True):
    with tf.name_scope("sub") :
        v=tf.get_variable("v", [1], initializer=tf.constant_initializer(5))
        w=tf.Variable(3, name="w")
        x = 1+v
        print(v.name)
        print(w.name)
        print(x.name)
        
#initializer가 제대로 안먹히는데,,,
        

namespace/v:0
namespace_11/sub/w:0
namespace_11/sub/add:0


# 시각화
anaconda cmd => tensorboard --logdir="/tmp/graphs" --port 6006 --host=127.0.0.1
실행 => 브라우저에서 localhost:6006


In [34]:
c1 = tf.constant(2)
c2 = tf.constant(3)
x = tf.add(c1, c2)
y = tf.subtract(c1,c2)

with tf.Session() as sess:
    sess.run(x)
    writer = tf.summary.FileWriter("/tmp/graphs", sess.graph)
    writer.close()

In [37]:
rnd1 = tf.random_uniform([1], 0, 100, dtype=tf.int32)
rnd2 = tf.random_uniform([1], 0, 100, dtype=tf.int32)
err = tf.subtract(rnd1[0] , rnd2[0])
err_summary = tf.summary.scalar('Error', err)
file_writer = tf.summary.FileWriter("/tmp/graphs")

with tf.Session() as sess:
    for step in range(1000):
        if step%10 == 0:
            summary_str = err_summary.eval()
            file_writer.add_summary(summary_str, step)
    file_writer.close()